In [ ]:
!pip install transformers
!pip install einops

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextGenerationPipeline
import torch
import textwrap

class MyTextProcessor:
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        self.pipeline = TextGenerationPipeline(
            model=self.model,
            tokenizer=self.tokenizer,
            device=0  # Use CPU. Change to the ID of your GPU if you have a GPU and want to use it.
        )

    def text_processing(self, transcription_path):
        with open(transcription_path, 'r') as f:
            text = f.read()
        text_fragments = textwrap.wrap(text, 3000)
        responses = []
        for fragment in text_fragments:
            prompt = f"""Actúa como un secretario de actas con experiencia, te daré un fragmento de texto de una reunión, quiero que me entregues un resumen en formato LaTeX que debe tener la siguiente estructura.\\section{{parte {text_fragments.index(fragment)+1}}} \\subsection{{Tema}} \\subsection{{Agenda}} \\subsection{{Decisiones}} \\subsection{{Compromisos}}, el contenido de la subsección Tema puedes ser un breve resumen, el contenido de la subsección Agenda debe ser los temas mas importantes que se mencionaron separados por comas, el contenido de la subsección Decisiones debe tener los temas de las decisiones mas importantes separados por comas y el contenido de la subsección Compromisos debe establecer los compromisos y si es posible con su responsable y fechas separados por comas, si no existe contenido en una subsección puedes escribir que no se establecieron en esa parte, no cambies el formato LaTeX establecido, no uses un entorno itimize, numerate o description,recuerda que lo que me entregues se colocará entre un \\begin{{document}} y \\end{{document}} que ya tiene su preámbulo Latex. {fragment}"""
            response = self.pipeline(prompt, max_length=300, do_sample=True)
            responses.append(response[0]['generated_text'])
        response_path = 'responses.txt'
        with open(response_path, 'w') as f:
            for response in responses:
                f.write(f"{response}\n")
        return response_path


In [ ]:
processor = MyTextProcessor("tiiuae/falcon-40b")
response_path = processor.text_processing("transcription.txt")